In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [34]:
df = pd.read_csv('train.csv')

In [35]:
df["date_time"] = pd.to_datetime(df["date_time"])
df["year"] = df["date_time"].dt.year
df["month"] = df["date_time"].dt.month

In [36]:
df.prop_location_score2.loc[np.isnan(df.prop_location_score2)] = 0

In [37]:
df.visitor_hist_starrating.loc[np.isnan(df.visitor_hist_starrating)] = 0
df.visitor_hist_adr_usd.loc[np.isnan(df.visitor_hist_adr_usd)] = 0


In [38]:
df.orig_destination_distance[np.isnan(df.orig_destination_distance)] = 0 
df.prop_review_score[np.isnan(df.prop_review_score)] = 0
df.srch_query_affinity_score[np.isnan(df.srch_query_affinity_score)] = 0


In [39]:
df.iloc[:,27:51] = df.iloc[:,27:51].fillna(value = 0, axis = 0)

In [40]:
df['prop_id_counts'] = df.groupby(['prop_id'])['prop_id'].transform('count')

In [41]:
df['usd_diff'] = abs(df.price_usd - df.visitor_hist_adr_usd)

In [42]:
df['star_diff'] = abs(df.prop_starrating - df.visitor_hist_starrating) 

In [43]:
df.prop_log_historical_price.loc[np.isnan(df.prop_log_historical_price)] = 0
df['diff_hist_price'] = np.exp(df.prop_log_historical_price) - df.price_usd


In [44]:
df['total_price'] = df.price_usd * df.srch_room_count

In [45]:
df['price_per_pers'] = (df.price_usd * df.srch_room_count)/(df.srch_adults_count + df.srch_children_count)

In [46]:
df['prop_score'] = (df.prop_location_score2 + 0.00001)/(df.prop_location_score1 + 0.00001)

In [47]:
df['nr_pers'] =  df.srch_adults_count + df.srch_children_count

In [48]:
df['price_rank'] = df.groupby('srch_id', sort=False)['price_usd'].rank(ascending=True, method ="min")
df['stars_rank'] = df.groupby('srch_id', sort=False)['prop_starrating'].rank(ascending=True, method = "min")
df['score_rank'] = df.groupby('srch_id', sort=False)['prop_location_score2'].rank(ascending=True, method = "min")


In [54]:
df['rank_rel'] = np.zeros(len(df))
df['rank_rel'] = np.where(df.click_bool == 1, 1, df.rank_rel)
df['rank_rel'] = np.where(df.booking_bool == 1, 5, df.rank_rel)

In [55]:
header_knn = ['prop_country_id','year', 'month', 'prop_score','prop_location_score1','prop_location_score2','diff_hist_price',
                'usd_diff', 'star_diff', 'srch_query_affinity_score', 'prop_review_score', 'prop_starrating', 'price_rank','stars_rank','score_rank', 'price_per_pers',
        'random_bool', 'price_usd', 'total_price', 'nr_pers','prop_id_counts', 'position', 'rank_rel']

header_regression = ['price_rank', 'stars_rank', 'score_rank', 'position', 'rank_rel']

In [56]:
df.to_csv('knn_train.csv', columns = header_knn)
df.to_csv('regression_train.csv', columns = header_regression)